In [1]:
import csv
import pandas as pd
import numpy as np
import scipy.linalg as linalg
from hodgerank_tools import *

In [2]:
symptoms = ['A', 'B', 'C','D']
#covid_data = np.array([symptoms, [3, 2, 2,5], [7, 8, 9, np.nan], [2, 2, 1,3]])
covid_df = pd.DataFrame(np.array([[3, 2, 2,5], [7, 8, 9, np.nan], [2, 2, 1,3]]), columns=symptoms)

In [3]:
symptoms = ['A', 'B', 'C','D']
my_data = df_to_dict_list(covid_df)
print(my_data)

[{'A': 3.0, 'B': 2.0, 'C': 2.0, 'D': 5.0}, {'A': 7.0, 'B': 8.0, 'C': 9.0}, {'A': 2.0, 'B': 2.0, 'C': 1.0, 'D': 3.0}]


In [4]:
my_nodes = get_nodes(my_data)
print(my_nodes)

['A', 'B', 'C', 'D']


In [5]:
get_f_W(my_data, my_nodes)

(array([0. , 0. , 1.5, 0. , 2. , 2.5]),
 array([[3., 0., 0., 0., 0., 0.],
        [0., 3., 0., 0., 0., 0.],
        [0., 0., 2., 0., 0., 0.],
        [0., 0., 0., 3., 0., 0.],
        [0., 0., 0., 0., 2., 0.],
        [0., 0., 0., 0., 0., 2.]]))

In [6]:
print(rank(my_data))

(  node         r
0    D  1.500000
1    A -0.409091
2    B -0.500000
3    C -0.590909, 0.38429752066115663)


In [7]:
print(naive_rank_0(my_data))

  node    r
0    A  4.0
1    B  4.0
2    C  4.0
3    D  4.0


In [8]:
# Ranks by the mean pairwise difference of each node
def naive_rank(data):
    #initialize nodes, sum_scores, total_votes
    nodes = get_nodes(data)
    (f, W) = get_f_W(data, nodes)
    sum_scores = dict()
    naive_r = dict()
    for node in nodes:
        sum_scores[node] = 0
    total_votes = sum_scores.copy()
    edges = get_edges(nodes)
    for node in nodes:
        # populate total_votes, sum_scores
        for i, edge  in enumerate(edges):
            if edge[0] == node:
                sum_scores[node] -= f[i]
            if edge[1] == node:
                sum_scores[node] += f[i]
            else:
                continue
            total_votes[node] += W[i, i]
        # calculate final score
        if total_votes[node] == 0:
            naive_r[node] = 0
        else:
            naive_r[node] = sum_scores[node]/total_votes[node]
    # format into a df
    rank_df = pd.DataFrame.from_dict(naive_r, orient = 'index', columns = ['r'])
    rank_df = rank_df.reset_index()
    rank_df = rank_df.rename(columns = {'index':'node'})
    rank_df = rank_df.sort_values(by =['r'],  ascending = False)
    rank_df = rank_df.reset_index(drop = True)
    #(f, W) = get_f_W(data, nodes)
    return (rank_df) #get_error(f, W, naive_r, nodes)
naive_rank(my_data)

,node,r
0,D,1.000000
1,A,0.000000
2,C,-0.416667
3,B,-0.666667


In [9]:
print(group_similar_scoring(my_data, 3))

[['D', 'A'], ['C'], ['B']]
